In [ ]:
import open3d as o3d
import numpy as np
from matplotlib import pyplot as plt
import os
from PIL import Image
import csv
import cv2

In [ ]:
ZED_data = '../../data/20221210/ZED/720/depth_PNG_3029_720_06-12-2022-19-39-53.png'
os.path.exists(ZED_data)

In [ ]:
zed_depth_image = Image.open(ZED_data).convert('I')
zed_depth = np.asarray(zed_depth_image)

In [ ]:
plt.imshow(zed_depth)
print(zed_depth.shape)

In [ ]:
LiDAR_data = '../../data/20221210/lidar/300/2022-12-10-16-53-36_Velodyne-VLP-16-Data.csv'
os.path.exists(LiDAR_data)

In [ ]:
with open(LiDAR_data, newline='') as f:
    rows = list(csv.reader(f, delimiter=',', quotechar='"'))
    sph_lidar = np.zeros(shape=(len(rows) - 1, 3))
    headers = rows.pop(0)
    for index, row in enumerate(rows[1:]):
        # print(index, row[0])
        sph_lidar[index] = row[7:10]

In [ ]:
indexer = {
    0:15,
    2:14,
    4:13,
    6:12,
    8:11,
    10:10,
    12:9,
    14:8,
    1:7,
    3:6,
    5:5,
    7:4,
    9:3,
    11:2,
    13:1,
    15:0,
}

sph_lidar[:,0] = np.array(list(map(indexer.get, sph_lidar[:,0])))

In [ ]:
mid_cutoff = 17100
sph_lidar[:,1] += mid_cutoff
mask = sph_lidar[:,1] > 2*mid_cutoff
sph_lidar[mask, 1] -= 2*mid_cutoff

In [ ]:
high_cutoff = mid_cutoff + 85 * 100 / 2
low_cutoff = mid_cutoff - 85 * 100 / 2

sph_lidar_85 = sph_lidar[sph_lidar[:,1] > low_cutoff]
sph_lidar_85 = sph_lidar_85[sph_lidar_85[:,1] < high_cutoff]

In [ ]:
num_arrays = 16

# Split the array based on the range of the first column
arrays = []
for i in range(num_arrays):
    lower = i
    upper = i + 1
    mask = (sph_lidar_85[:, 0] >= lower) & (sph_lidar_85[:, 0] < upper)
    sub_array = sph_lidar_85[mask, 1:]
    arrays.append(sub_array)

# Convert each sub-array to a 1D array sorted by the first column and using the second column as the value
sph_lidar_frame = []
for sub_array in arrays:
    indices = np.argsort(sub_array[:, 0])
    sorted_array = sub_array[indices, 1].tolist()

    row_len = len(sorted_array)
    num_zeros = 1280 - row_len
    step_size = row_len // (num_zeros + 1)

    # Loop over the array and insert zeros at regular intervals
    for i in range(num_zeros):
        index = (i + 1) * step_size
        sorted_array.insert(index, 0)
    print(len(sorted_array))
    
    sph_lidar_frame.append(sorted_array)
sph_lidar_frame = np.asarray(sph_lidar_frame)

In [ ]:
print(sph_lidar_frame.shape)

In [ ]:
m = 400//16
gt_mean = sph_lidar_85[:,2].mean()

us = zed_depth
us = us / 1000  # us.mean() * gt_mean

In [ ]:
# print(np.count_nonzero(us > 20), (zed_depth > 20).size)
# us = us / us.mean() * gt_mean


In [ ]:
us[160:560,:][::m,:] = sph_lidar_frame
print(np.count_nonzero(us))

plt.imshow(us)

In [ ]:
us[us > 20] = 20
us[us < 1] = 1

In [ ]:
print(gt_mean, us.mean())
print(us.min(),us.max())

In [ ]:
print(us.shape)
print(us[160:560,:][::m,:].shape)

In [ ]:
def lpf(img, ncutoff):
    # Apply 2D FFT to the image
    f = np.fft.fft2(img)

    # Shift the zero frequency component to the center of the spectrum
    fshift = np.fft.fftshift(f)

    # Create a circular mask of the same size as the spectrum
    rows, cols = img.shape
    crow, ccol = rows // 2, cols // 2
    mask = np.zeros((rows, cols), np.uint8)
    cutoff = int(min(crow, ccol)*ncutoff)
    axes = (5, 10)
    cv2.circle(mask, (ccol, crow), cutoff, 1, -1)
    # cv2.ellipse(mask, (ccol, crow), axes, 0, 0, 360,  1, -1)

    # Apply the mask to the shifted spectrum
    fshift_filtered = fshift * mask

    # Shift the zero frequency component back to the corner of the spectrum
    f_filtered = np.fft.ifftshift(fshift_filtered)

    # Apply the inverse 2D FFT to the filtered spectrum
    img_filtered = np.fft.ifft2(f_filtered)
    img_filtered = np.real(img_filtered)

    return img_filtered

def brf(img, low_cutoff, high_cutoff):
    # Apply 2D FFT to the image
    f = np.fft.fft2(img)

    # Shift the zero frequency component to the center of the spectrum
    fshift = np.fft.fftshift(f)

    # Create a circular mask of the same size as the spectrum
    rows, cols = img.shape
    crow, ccol = rows // 2, cols // 2
    mask_low = np.zeros((rows, cols), np.uint8)
    mask_high = np.zeros((rows, cols), np.uint8)

    # Create two circular masks with different radii
    low_cutoff = int(min(crow, ccol)*low_cutoff)
    high_cutoff = int(min(crow, ccol)*high_cutoff)
    cv2.circle(mask_low, (ccol, crow), low_cutoff, 1, -1)
    cv2.circle(mask_high, (ccol, crow), high_cutoff, 1, -1)

    # Combine the two masks to create a bandpass filter
    mask = cv2.bitwise_xor(mask_low, mask_high)

    # Invert the filter to create a band reject filter
    mask = 1 - mask

    # Apply the mask to the shifted spectrum
    fshift_filtered = fshift * mask

    # Shift the zero frequency component back to the corner of the spectrum
    f_filtered = np.fft.ifftshift(fshift_filtered)

    # Apply the inverse 2D FFT to the filtered spectrum
    img_filtered = np.fft.ifft2(f_filtered)
    img_filtered = np.real(img_filtered)

    return img_filtered

def pg(input, us_rate, gt_mean, threshold = 100, lcutoff = 0.2, hcutoff = 0.4):
# def pg(input, us_rate, ncutoff, gt_mean, iter):

    pg_mean = 0
    filtered = input
    
    while (pg_mean > gt_mean * 1.1 or pg_mean < gt_mean * 0.9   ) and threshold > 0:
    # for i in range(iter):
        filtered = lpf(filtered, lcutoff)
        # filtered = brf(filtered, low_cutoff=lcutoff, high_cutoff=hcutoff)
        filtered[::us_rate, ::us_rate] = input[::us_rate, ::us_rate]
        pg_mean = filtered.mean()
        threshold -=1
    
    return filtered

In [ ]:
pg_frame = us
pg_frame = pg(us, m, gt_mean=gt_mean, threshold=50, lcutoff=0.1, hcutoff=0.3)
plt.imshow(pg_frame)

In [ ]:
def back_to_pts_form(arr):
    # get the shape of the input array
    m, n = arr.shape
    azimuth_const = 85/n
    polar_const = 30/m
    
    # create a 3D output array of size (m * n, 3)
    out = np.zeros((m * n, 3))
    
    # populate the output array
    for row in range(m):
        for col in range(n):
            index = row * n + col
            out[index, 0] = arr[row, col]
            out[index, 1] = row * polar_const
            out[index, 2] = col * azimuth_const 
    
    return out

In [ ]:
back_pts = back_to_pts_form(pg_frame)
# back_pts[:,1] = np.array(list(map(angle.get, back_pts[:,1])))
back_pts[:,1] = np.radians(back_pts[:,1])
back_pts[:,2] = np.radians(back_pts[:,2])

In [ ]:
point_cloud_data = back_pts

# Convert spherical coordinates to Cartesian coordinates
x = point_cloud_data[:, 0] * np.cos(point_cloud_data[:, 1]) * np.cos(point_cloud_data[:, 2])
y = point_cloud_data[:, 0] * np.cos(point_cloud_data[:, 1]) * np.sin(point_cloud_data[:, 2])
z = point_cloud_data[:, 0] * np.sin(point_cloud_data[:, 1])

pg_pts = np.asarray([x, y, z]).T

pcd_pg_lidar = o3d.geometry.PointCloud()
pcd_pg_lidar.points = o3d.utility.Vector3dVector(pg_pts)

In [ ]:
sph_lidar[:,0] = np.radians(sph_lidar[:,0])
sph_lidar[:,1] = np.radians(sph_lidar[:,1] / 100)

point_cloud_data = sph_lidar

# Convert spherical coordinates to Cartesian coordinates
x = point_cloud_data[:, 2] * np.cos(point_cloud_data[:, 0]) * np.cos(point_cloud_data[:, 1])
y = point_cloud_data[:, 2] * np.cos(point_cloud_data[:, 0]) * np.sin(point_cloud_data[:, 1])
z = point_cloud_data[:, 2] * np.sin(point_cloud_data[:, 0])

inp_pts = np.asarray([x, y, z]).T

pcd_inp_lidar = o3d.geometry.PointCloud()
pcd_inp_lidar.points = o3d.utility.Vector3dVector(inp_pts)

In [ ]:
o3d.visualization.draw_geometries([
    pcd_pg_lidar, 
    # pcd_inp_lidar
    ])